# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
source_path = "..\weatherpy\output_data\cities.csv"

cities_df = pd.read_csv(source_path)
cities_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,-34.4187,19.2345,79.81,70.0,70.0,11.99,ZA,1.647768e+09
1,1,tuktoyaktuk,69.4541,-133.0374,-23.93,100.0,92.0,5.68,CA,1.647768e+09
2,2,dikson,73.5069,80.5464,6.71,96.0,74.0,8.57,RU,1.647768e+09
3,3,saquarema,-22.9200,-42.5103,73.44,86.0,98.0,3.89,BR,1.647768e+09
4,4,kristiansund,63.1115,7.7320,45.25,70.0,0.0,2.30,NO,1.647768e+09
...,...,...,...,...,...,...,...,...,...,...
613,613,payson,40.0444,-111.7321,50.83,38.0,75.0,9.22,US,1.647768e+09
614,614,chortomlyk,47.6232,34.1420,38.10,36.0,4.0,11.68,UA,1.647769e+09
615,615,valentin gomez farias,28.2167,-106.5833,48.22,29.0,0.0,1.07,MX,1.647769e+09
616,616,boden,65.8252,21.6886,46.40,66.0,100.0,7.29,SE,1.647769e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# check data completeness
cities_df = cities_df.dropna()
cities_df.count()

Unnamed: 0    566
City          566
Lat           566
Lng           566
Max Temp      566
Humidity      566
Cloudiness    566
Wind Speed    566
Country       566
Date          566
dtype: int64

In [4]:
# Configure API key for Gmap
gmaps.configure(api_key=g_key)

# Determine cities location based on Lat and Long
locations_cities = cities_df[["Lat","Lng"]].astype(float)

# Determine humidity value
humidity = cities_df["Humidity"].astype(float)

# Create humidity heatmap layer
fig_humid = gmaps.figure()
humid_heat_layer = gmaps.heatmap_layer(locations_cities,weights = humidity, dissipating=False, max_intensity=50, point_radius=1)
fig_humid.add_layer(humid_heat_layer)
fig_humid



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
# create dataframe to narrow down the cities
vacation_cities_df = cities_df.loc[(cities_df["Max Temp"] > 70) & 
(cities_df["Max Temp"] < 80) & 
(cities_df["Wind Speed"] < 10) & 
(cities_df["Cloudiness"] < 10)]

# check number of filtered cities
vacation_cities_df.count()

Unnamed: 0    8
City          8
Lat           8
Lng           8
Max Temp      8
Humidity      8
Cloudiness    8
Wind Speed    8
Country       8
Date          8
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [39]:
# create hotel_df: recut previous datafram and add hotel name column
hotel_df = vacation_cities_df[["City","Country","Lat","Lng"]]
hotel_df = hotel_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,arraial do cabo,BR,-22.9661,-42.0278,
1,east london,ZA,-33.0153,27.9116,
2,vila velha,BR,-20.3297,-40.2925,
3,moranbah,AU,-22.0016,148.0466,
4,huixtla,MX,15.1500,-92.4667,
5,sao joao da barra,BR,-21.6403,-41.0511,
6,isla mujeres,MX,21.2311,-86.7310,
7,piacabucu,BR,-10.4056,-36.4344,


In [42]:
#create parameter dictionary
parameter = {"types": "lodging",
    "radius" : 5000,
    "business_status" : "OPERATIONAL",
    "permanently_closed": False,
    "key": g_key}

# loop through the chosen cities
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    hotel_lat = row["Lat"]
    hotel_lng = row["Lng"]

    # update location parameters
    parameter["location"] = f"{hotel_lat},{hotel_lng}"

    hotel_name = requests.get(base_url,params=parameter)

    # convert to kson
    hotel_name = hotel_name.json()

    hotel_df.loc[index,"Hotel Name"] = hotel_name["results"][0]["name"]

hotel_df

,City,Country,Lat,Lng,Hotel Name
0,arraial do cabo,BR,-22.9661,-42.0278,Pousada Porto Praia
1,east london,ZA,-33.0153,27.9116,Tu Casa
2,vila velha,BR,-20.3297,-40.2925,Hotel Vitória Palace
3,moranbah,AU,-22.0016,148.0466,Direct Hotels - Monterey Moranbah
4,huixtla,MX,15.1500,-92.4667,Rocky Point Sonoran Sky
5,sao joao da barra,BR,-21.6403,-41.0511,Pousada Porto De Canoas
6,isla mujeres,MX,21.2311,-86.7310,Hotel La Joya Isla Mujeres
7,piacabucu,BR,-10.4056,-36.4344,Piaçabuçu News


In [44]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_hotels = hotel_df[["Lat", "Lng"]]

In [48]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    locations_hotels,
    info_box_content=hotel_info)



# Display figure
fig_humid = gmaps.figure()
fig_humid.add_layer(humid_heat_layer)
fig_humid.add_layer(hotel_layer)
fig_humid

Figure(layout=FigureLayout(height='420px'))